In [38]:
from Speaker_identification import Speaker_identification
from density_func import density_func, naive_G_U
from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from process_bar import ShowProcess
from scipy.io import loadmat,savemat

'''
Import features
'''

#save_path="D:\\LAB\\lab\\task_2_version_4\\features.txt"
save_path="/Users/Mata/Documents/lab/task_2_version_4/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

#save_path="D:\\LAB\\lab\\task_2_version_4\\unknown_features.txt"
save_path="/Users/Mata/Documents/lab/task_2_version_4/unknown_feature.txt"
f = open(save_path,'rb')
b_new_test=pickle.load(f)
#print(b_new_test.shape)
f.close()

'''
Import UBM model
'''

ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz",mat_dtype=True)
#ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
start=time.time()
error_set={}  
num_samples=len(features.keys())
name_set=list(features.keys())
name_set.append("unknown")
correct_sum=0
false_sum=0
for cross_num in range(1):
    #process_bar.show_process()
    train_file_set=[]
    test_file_set=[]
    correct_num=0
    false_num=0
    '''
    Split the test and train set
    '''
    for name in features.keys():
        whole_set=features.get(name,'no such file').copy()
        test_file=whole_set[cross_num]
        train_num=list(range(10))
        train_num.remove(cross_num)
        test_file_set.append(test_file)
        #name_set.append(name)
        train_set=[]
        for num in train_num:
            train_set.append(whole_set[num])
        train_set=np.concatenate(train_set,axis=1)
        train_file_set.append(train_set)
        
    test_file_set.append(b_new_test)
    
    '''
    Start modeling and identification
    '''
    print("crossvalidation "+str(cross_num+1)+" start")
    process_bar_2=ShowProcess(num_samples)
    scores_set=np.zeros((num_samples+1,num_samples+1))
    for index_2 in range(num_samples):
        process_bar_2.show_process()
        b_train=train_file_set[index_2]
        gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=1,weights_init=ubm_weights,\
                            means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
        gmm.fit(b_train.T)
        for index_1 in range(num_samples+1):
            #print("now test set "+str(index_1)+" is testing "+str(index_2))
            b_test=np.array(test_file_set[index_1])
            scores_set[index_1,index_2]=gmm.score(b_test.T)
    '''
    Enhancement : added the unknown detection part
    '''
    for index in range(num_samples+1):
        b_test=np.array(test_file_set[index])
        T_value=b_test.shape[1]
        unknown_score=Speaker_identification(b_test,ubm_means,ubm_var,ubm_weights,T_value)
        #scores_set[index,num_samples]=unknown_score
        scores_set[index,num_samples]=(1/T_value)*unknown_score
        
        
    '''
    Calculate the detection rate
    '''
    for index in range(num_samples+1):
        test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
        if index == test_index:
            correct_num +=1
            correct_sum +=1
        else:
            false_num +=1
            false_sum +=1
            print('')
            print("error ! True: "+name_set[index]+" False: "+name_set[test_index])
        #print("time cost %5.1f second"%((time.time()-start)/60))

    process_bar_2.close()

    detection_rate=correct_num/(correct_num+false_num)
    print("crossvalidation "+str(cross_num+1)+" compeleted")
    print("cost time %5.1f minute"%((time.time()-start)/60))
    detection_rate=correct_num/(false_num+correct_num)
    detection_rate_set.append(detection_rate)
    print("the crossval "+str(cross_num+1)+" detection_rate is "+str(detection_rate))

print("the total detection rate is ",correct_sum/(correct_sum+false_sum))



crossvalidation 1 start


/Users/Mata/anaconda3/lib/python3.6/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
error ! True: unknown False: fawf0
done
crossvalidation 1 compeleted
cost time   4.9 minute
the crossval 1 detection_rate is 0.9940828402366864
the total detection rate is  0.9940828402366864


In [40]:
scores_set[168,:]

array([-35.51519603, -38.14051534, -38.2124678 , -34.07467284,
       -36.40435531, -38.45074757, -42.89576008, -36.17503165,
       -38.53905622, -40.91032702, -35.1482316 , -37.10099576,
       -35.8032146 , -34.16160678, -34.69825106, -35.25324757,
       -35.81697615, -34.77595606, -35.58704425, -37.28708217,
       -41.44640222, -37.90797328, -37.1365751 , -38.45573761,
       -39.51095343, -38.58893046, -37.67201474, -36.18494951,
       -45.30189201, -36.37004475, -38.75490811, -39.2677652 ,
       -37.7314581 , -39.67996619, -37.16377568, -37.73528716,
       -37.77605806, -35.71307186, -34.48734917, -34.04559459,
       -38.61096439, -44.06327109, -38.64847742, -36.63698315,
       -38.5497419 , -36.84171487, -36.30353075, -38.78994132,
       -34.40807189, -36.27177903, -36.96216355, -37.42281945,
       -38.01592701, -41.26351049, -37.29384382, -37.94768486,
       -39.50812816, -38.21649956, -36.91731616, -38.08729469,
       -37.72873642, -37.49673207, -40.16858659, -35.33

In [7]:

from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from process_bar import ShowProcess
from scipy.io import loadmat,savemat

'''
Import features
'''

save_path="D:\\LAB\\lab\\task_2_version_4\\features.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_4/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

save_path="D:\\LAB\\lab\\task_2_version_4\\unknown_features.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_4/unknown_features.txt"
f = open(save_path,'rb')
unknown_features=pickle.load(f)  #190 unregisterd user
#print(b_new_test.shape)
f.close()

'''
Import UBM model
'''

#ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz",mat_dtype=True)
ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
start=time.time()
error_set={}  
num_registered=len(features.keys())
num_unregistered=len(unknown_features.keys())
name_set=list(features.keys())
#name_set.append("unknown")
correct_sum=0
false_sum=0
ml_error_sum=0
fr_error_sum=0
fa_error_sum=0
threshold=-34.8
for cross_num in range(1):
    #process_bar.show_process()
    train_file_set=[]
    test_file_set=[]
    correct_num=0
    false_num=0
    ml_error=0
    fr_error=0
    fa_error=0
    '''
    Split the test and train set
    '''
    for name in features.keys():
        whole_set=features.get(name,'no such file').copy()
        test_file=whole_set[cross_num]
        train_num=list(range(10))
        train_num.remove(cross_num)
        test_file_set.append(test_file)
        #name_set.append(name)
        train_set=[]
        for num in train_num:
            train_set.append(whole_set[num])
        train_set=np.concatenate(train_set,axis=1)
        train_file_set.append(train_set)
        
    for name in unknown_features.keys():
        whole_set=unknown_features.get(name,'no such file').copy()
        test_file=whole_set[cross_num]
        test_file_set.append(test_file)

    
    '''
    Start modeling and identification
    '''
    print("crossvalidation "+str(cross_num+1)+" start")
    process_bar_2=ShowProcess(num_registered)
    scores_set=np.zeros((num_registered+num_unregistered,num_registered))
    for index_2 in range(num_registered):
        process_bar_2.show_process()
        b_train=train_file_set[index_2]
        gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=1,weights_init=ubm_weights,\
                            means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
        gmm.fit(b_train.T)
        for index_1 in range(num_registered+num_unregistered):
            #print("now test set "+str(index_1)+" is testing "+str(index_2))
            b_test=np.array(test_file_set[index_1])
            scores_set[index_1,index_2]=gmm.score(b_test.T)

        
    '''
    Calculate the detection rate and error rate
    '''
    for index in range(num_registered+num_unregistered):
        if index < num_registered :
            if max(scores_set[index,:])>threshold*ubm:
                test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
                if index == test_index:
                    correct_num +=1
                    correct_sum +=1
                else:
                    false_num +=1
                    false_sum +=1
                    ml_error +=1
                    ml_error_sum +=1
                    print('')
                    print("error ! True: "+name_set[index]+" False: "+name_set[test_index])
            else:
                    false_num+=1
                    false_sum+=1
                    fr_error+=1
                    fr_error_sum+=1
                    print('')
                    print("error ! True: "+name_set[index]+" False: unknowns")
        else:
            if max(scores_set[index,:])<=threshold:
                correct_num +=1
                correct_sum +=1
            else:
                false_num+=1
                false_sum+=1
                fa_error+=1
                fa_error_sum+=1
                print('')
                print("error ! Unknown misidentified")
                
            
        #print("time cost %5.1f second"%((time.time()-start)/60))

    process_bar_2.close()

    detection_rate=correct_num/(correct_num+false_num)
    print("crossvalidation "+str(cross_num+1)+" compeleted")
    print("cost time %5.1f minute"%((time.time()-start)/60))
    detection_rate=correct_num/(false_num+correct_num)
    ml_rate=100*ml_error/(num_registered)
    fr_rate=100*fr_error/(num_registered)
    fa_rate=100*fa_error/(num_unregistered)
    aer_rate=100*(ml_error+fr_error+fa_error)/(num_registered+num_unregistered)
    #detection_rate_set.append(detection_rate)
    print("the crossval "+str(cross_num+1)+" detection_rate is "+str(detection_rate))
    print("ML %5.1f, FR, %5.1f , FA, %5.1f, AER, %5.1f"%(ml_rate,fr_rate,fa_rate,aer_rate))
print("the total detection rate is ",correct_sum/(correct_sum+false_sum))
print("the total detection rate is ",correct_sum/(correct_sum+false_sum))



crossvalidation 1 start


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Unknown misidentified

error ! Un

In [11]:
from Speaker_identification import Speaker_identification
from density_func import density_func, naive_G_U
from sklearn.mixture import GaussianMixture
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
#default

import pickle
import numpy as np
import time

#loadmat
from process_bar import ShowProcess
from scipy.io import loadmat,savemat

'''
Import features
'''

save_path="D:\\LAB\\lab\\task_2_version_4\\features.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_4/features.txt"
f = open(save_path,'rb')
features=pickle.load(f)
f.close()

save_path="D:\\LAB\\lab\\task_2_version_4\\unknown_features.txt"
#save_path="/Users/Mata/Documents/lab/task_2_version_4/unknown_features.txt"
f = open(save_path,'rb')
unknown_features=pickle.load(f)  #190 unregisterd user
#print(b_new_test.shape)
f.close()

'''
Import UBM model
'''

#ubm_dataset=loadmat("/Users/Mata/Documents/2017/学习/ws2017:18/PUL/forStudents/ubm/UBM_GMMNaive_MFCC_Spectrum0to8000Hz",mat_dtype=True)
ubm_dataset=loadmat("C:\\Users\\hasee\\workspace\\workspace\\lab\\patRecDat\\forStudents\\ubm\\UBM_GMMNaive_MFCC_Spectrum0to8000Hz.mat",mat_dtype=True)
ubm_means=ubm_dataset['means']
ubm_var = ubm_dataset['var']
ubm_weights = ubm_dataset['weights'].ravel()
ubm_var_set=[]
K_value=49
gamma_UBM=1

#transfer variance of UBM to cov
for k in range(K_value):
    ubm_var_set.append(np.diag(ubm_var[k]))
ubm_var_set=np.array(ubm_var_set)
ubm_var=ubm_var_set

num_people=len(features.keys())

'''
Start Crossvalidation
'''
#process_bar=ShowProcess(10)
detection_rate_set=[]
ml_rate_set=[]
fr_rate_set=[]
fa_rate_set=[]
aer_rate_set=[]
start=time.time()
error_set={}  
num_registered=len(features.keys())
num_unregistered=len(unknown_features.keys())
name_set=list(features.keys())
#name_set.append("unknown")
correct_sum=0
false_sum=0
ml_error_sum=0
fr_error_sum=0
fa_error_sum=0
for threshold in [0.3]:
    #process_bar.show_process()
    train_file_set=[]
    test_file_set=[]
    correct_num=0
    false_num=0
    ml_error=0
    fr_error=0
    fa_error=0
    '''
    Split the test and train set
    '''
    for name in features.keys():
        whole_set=features.get(name,'no such file').copy()
        test_file=whole_set[0]
        train_num=list(range(10))
        train_num.remove(0)
        test_file_set.append(test_file)
        #name_set.append(name)
        train_set=[]
        for num in train_num:
            train_set.append(whole_set[num])
        train_set=np.concatenate(train_set,axis=1)
        train_file_set.append(train_set)
        
    for name in unknown_features.keys():
        whole_set=unknown_features.get(name,'no such file').copy()
        test_file=whole_set[0]
        test_file_set.append(test_file)

    
        '''
    Enhancement : added the unknown detection part
    '''
    ubm_scores=[]
    for index in range(num_registered+num_unregistered):
        b_test=np.array(test_file_set[index])
        T_value=b_test.shape[1]
        unknown_score=Speaker_identification(b_test,ubm_means,ubm_var,ubm_weights,T_value)
        #scores_set[index,num_samples]=unknown_score
        ubm_scores.append((1/T_value)*unknown_score)
        
    '''
    Start modeling and identification
    '''
    #print("crossvalidation "+str(cross_num+1)+" start")
    process_bar_2=ShowProcess(num_registered)
    scores_set=np.zeros((num_registered+num_unregistered,num_registered))
    for index_2 in range(num_registered):
        process_bar_2.show_process()
        b_train=train_file_set[index_2]
        gmm=GaussianMixture(n_components=K_value,covariance_type='full',max_iter=1,weights_init=ubm_weights,\
                            means_init=ubm_means,precisions_init=np.linalg.inv(ubm_var))
        gmm.fit(b_train.T)
        for index_1 in range(num_registered+num_unregistered):
            #print("now test set "+str(index_1)+" is testing "+str(index_2))
            b_test=np.array(test_file_set[index_1])
            scores_set[index_1,index_2]=gmm.score(b_test.T)

        
    '''
    Calculate the detection rate and error rate
    '''
    for index in range(num_registered+num_unregistered):
        if index < num_registered :
            if max(scores_set[index,:])>threshold*ubm_scores[index]:
                test_index=np.int(np.argwhere(scores_set[index,:]==max(scores_set[index,:])))
                if index == test_index:
                    correct_num +=1
                    correct_sum +=1
                else:
                    false_num +=1
                    false_sum +=1
                    ml_error +=1
                    ml_error_sum +=1
                    print('')
                    print("error ! True: "+name_set[index]+" False: "+name_set[test_index])
            else:
                    false_num+=1
                    false_sum+=1
                    fr_error+=1
                    fr_error_sum+=1
                    print('')
                    print("error ! True: "+name_set[index]+" False: unknowns")
        else:
            if max(scores_set[index,:])<=threshold*ubm_scores[index]:
                correct_num +=1
                correct_sum +=1
            else:
                false_num+=1
                false_sum+=1
                fa_error+=1
                fa_error_sum+=1
                print('')
                print("error ! Unknown misidentified")
                
            
        #print("time cost %5.1f second"%((time.time()-start)/60))

    process_bar_2.close()

    detection_rate=correct_num/(correct_num+false_num)
    print("threshold "+str(threshold)+" compeleted")
    print("cost time %5.1f minute"%((time.time()-start)/60))
    detection_rate=correct_num/(false_num+correct_num)
    ml_rate=100*ml_error/(num_registered)
    fr_rate=100*fr_error/(num_registered)
    fa_rate=100*fa_error/(num_unregistered)
    aer_rate=100*(ml_error+fr_error+fa_error)/(num_registered+num_unregistered)
    ml_rate_set.append(ml_rate)
    fr_rate_set.append(fr_rate)
    fa_rate_set.append(fa_rate)
    aer_rate_set.append(aer_rate)
    #detection_rate_set.append(detection_rate)
    print("threshold "+str(threshold)+" detection_rate is "+str(detection_rate))
    print("ML %5.1f, FR, %5.1f , FA, %5.1f, AER, %5.1f"%(ml_rate,fr_rate,fa_rate,aer_rate))
ml_rate_sum=100*ml_error/(num_registered*18)
fr_rate_sum=100*fr_error/(num_registered*18)
fa_rate_sum=100*fa_error/(num_unregistered*18)
aer_rate_sum=100*(ml_error+fr_error+fa_error)/((num_registered+num_unregistered)*18)
print("the total detection rate is ",correct_sum/(correct_sum+false_sum))
print("the Toal ML %5.1f, FR, %5.1f , FA, %5.1f, AER, %5.1f"%(ml_rate_sum,fr_rate_sum,fa_rate_sum,aer_rate_sum))



ModuleNotFoundError: No module named 'density_func'

-34.5477372316


In [29]:
maxscores_set[167,:]

array([-35.16739576, -35.07816931, -38.01358023, -34.55097793,
       -35.14696366, -35.6207894 , -34.7130317 , -35.01925468,
       -34.33254802, -36.27045808, -34.61785226, -35.49230568,
       -37.69402409, -34.25104765, -36.31212361, -35.97375522,
       -35.50952335, -35.15286143, -33.74885409, -34.49444825,
       -35.35892987, -34.60846175, -34.97539477, -35.13104644,
       -36.96582448, -34.91243364, -35.47891614, -33.80781995,
       -36.25763411, -34.23478348, -35.06876325, -35.76196391,
       -35.07293055, -34.67811964, -34.10971356, -36.09592121,
       -34.68701369, -36.39633513, -36.18863364, -34.87000502,
       -36.24206897, -36.7490916 , -35.00100171, -33.58947464,
       -35.06855828, -33.68204238, -34.52316123, -34.99981336,
       -34.15786625, -33.24105488, -34.12014232, -33.38512227,
       -34.57652049, -36.21805711, -35.75220225, -35.17655419,
       -35.31555152, -34.64501274, -34.64348095, -35.91898606,
       -34.30680853, -33.87006891, -35.15421479, -35.60